## Install necessary libraries

In [ ]:
pip install transformers shap datasets

## Import libraries

In [ ]:
import datasets
import numpy as np
import transformers
import shap

## Load dataset

In [ ]:
dataset = datasets.load_dataset("imdb", split="test")

# shorten the strings to fit into the pipeline model
short_data = [v[:500] for v in dataset["text"][:20]]

In [ ]:
print(dataset[0]["text"])

In [ ]:
print(short_data[0])

## Display the dataset

In [ ]:
import pandas as pd

# Create a pandas dataframe
df = pd.DataFrame(short_data)

# Display the head of the dataframe
print(df)

## Defining the NLP pipeline

In [ ]:
classifier = transformers.pipeline("sentiment-analysis", return_all_scores=True)

In [ ]:
# Example texts for sentiment analysis
texts = ["I love this movie! It was the best movie of all time", "I liked the first half but did not like the second half."]

# Perform sentiment analysis
results = classifier(texts)

print(results)

## Sentiment Analysis for first few data points

In [ ]:
print("1: ", short_data[0],"\n")
print("2: ", short_data[1],"\n")
print("3: ",short_data[2],"\n")
print("\n")
print(classifier(short_data[:3]))

## Find the model accuracy

In [ ]:
from sklearn.metrics import accuracy_score

# Predict the sentiment for the short data
predictions = classifier(short_data)

# Convert the predictions to labels
predicted_labels = []
for prediction in predictions:
    predicted_label = max(prediction, key=lambda x: x['score'])['label']
    predicted_labels.append(0 if predicted_label == 'NEGATIVE' else 1)  # Assuming 'NEGATIVE' is 0 and 'POSITIVE' is 1

true_labels = dataset["label"][:20]  # Get the true labels

# Calculate the accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

print(true_labels)

print(predicted_labels)

In [ ]:
print(dataset[12500])

## Define SHAP explainer

In [ ]:
# define the explainer
explainer = shap.Explainer(classifier)

## Explain the predictions on first 2 samples

In [ ]:
# explain the predictions of the pipeline on the first two samples
shap_values = explainer(short_data[:2])

## SHAP force plot and text plot

In [ ]:
print(shap_values.shape)

In [ ]:
shap.plots.text(shap_values[:, :, "POSITIVE"])

## Let us create a transformer pipeline wrapper

In [ ]:
pmodel = shap.models.TransformersPipeline(classifier, rescale_to_logits=False)

In [ ]:
pmodel(short_data[:2])

In [ ]:
pmodel = shap.models.TransformersPipeline(classifier, rescale_to_logits=True)
pmodel(short_data[:2])

In [ ]:
explainer2 = shap.Explainer(pmodel)
shap_values2 = explainer2(short_data[:2])
shap.plots.text(shap_values2[:, :, 1])

## Tokenizer as masker object

In [ ]:
masker = shap.maskers.Text(classifier.tokenizer)
explainer3 = shap.Explainer(pmodel, masker)
shap_values3 = explainer3(short_data[:2])
shap.plots.text(shap_values3[:, :, 1])

## Build a text masker

In [ ]:
masker = shap.maskers.Text(classifier.tokenizer)
explainer2 = shap.Explainer(pmodel, masker)
shap_values2 = explainer2(short_data[:2])
shap.plots.text(shap_values2[:, :, 1])

## Explore how the Text masker works

In [ ]:
masker.shape("I like this movie.")

In [ ]:
model_args = masker(
    np.array([True, True, True, True, True, True, True]), "I like this movie."
)
model_args

In [ ]:
pmodel(*model_args)

In [ ]:
model_args = masker(
    np.array([True, True, False, False, True, True, True]), "I like this movie."
)
model_args

In [ ]:
pmodel(*model_args)

In [ ]:
masker2 = shap.maskers.Text(
    classifier.tokenizer, mask_token="...", collapse_mask_token=True
)

In [ ]:
model_args2 = masker2(
    np.array([True, True, False, False, True, True, True]), "I like this movie."
)
model_args2

In [ ]:
pmodel(*model_args2)

## Plot summary statistics and bar charts

In [ ]:
# explain the predictions of the pipeline on the first two samples
shap_values = explainer(short_data[:20])

In [ ]:
shap.plots.bar(shap_values[0, :, "POSITIVE"])

In [ ]:
shap.plots.bar(shap_values[:, :, "POSITIVE"].mean(0))

In [ ]:
shap.plots.bar(shap_values[:, :, "POSITIVE"].mean(0), order=shap.Explanation.argsort)